In [389]:
SEARCH_WORD=input()

 Target


In [390]:
kw_list = [SEARCH_WORD]
STATE = "MN"

## Connecting to Google API to retrive search popularity and related terms

In [391]:
from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt
import os
import tweepy as tw
from textblob import TextBlob

In [392]:
pytrends = TrendReq(hl='en-US', tz=360)
pytrends.build_payload(kw_list, cat=0, timeframe='today 1-m', geo = 'US-MN', gprop='')

In [393]:
topics_by_region = pytrends.interest_by_region(resolution='CITY', inc_geo_code=False, inc_low_vol=True).reset_index()

In [394]:
topics_by_region

,geoName,Target
0,Duluth MN-Superior WI,56
1,Fargo-Valley City ND,23
2,La Crosse-Eau Claire WI,11
3,Mankato MN,70
4,Minneapolis-St. Paul MN,100
5,Rochester MN-Mason City IA-Austin MN,69


In [395]:
related_queries = pytrends.related_queries()
related_queries

{'Target': {'rising':                                 query   value
  0                 target cyber monday  100050
  1           target black friday hours   72200
  2           target thanksgiving hours   55550
  3                 target eden prairie   27500
  4   target hours on thanksgiving 2019   21700
  5             target weighted blanket   21500
  6               target christmas tree   16550
  7               target gift card sale   16500
  8                             bestbuy   11100
  9                   target northfield   11100
  10                       shipt target   10900
  11                 target photo print   10850
  12     is target open on thanksgiving    5450
  13                   target winona mn    5350
  14               walmart black friday    2500
  15      target black friday sale 2019    1250
  16              best buy black friday    1150
  17          walmart black friday 2019    1050
  18            walmart black friday ad     950
  19                

In [396]:
top_related_queries = related_queries[SEARCH_WORD]['top'].sort_values(by='value', ascending=False)
list_top_related_queries = top_related_queries["query"].values
list_top_related_queries

array(['target black friday', 'target hours', 'black friday target 2019',
       'target ad', 'target center', 'target near me',
       'walmart black friday', 'target black friday ad 2019',
       'target black friday ad', 'target pharmacy', 'target cyber monday',
       'target registry', 'target black friday sale 2019',
       'target black friday hours', 'target gift card',
       'cvs pharmacy target', 'best buy black friday',
       'target black friday deals', 'walmart black friday 2019',
       'walmart black friday ad', 'target red card',
       'target thanksgiving hours', 'target registry baby',
       'black friday deals', 'target redcard'], dtype=object)

In [397]:
interest_over_time= pytrends.interest_over_time().reset_index()
interest_over_time

,date,Target,isPartial
0,2019-11-09,40,False
1,2019-11-10,42,False
2,2019-11-11,30,False
3,2019-11-12,24,False
4,2019-11-13,26,False
5,2019-11-14,29,False
6,2019-11-15,30,False
7,2019-11-16,38,False
8,2019-11-17,38,False
9,2019-11-18,31,False


## Connecting to twitter database to retrive tweets

In [398]:
# Go to http://apps.twitter.com and create an app.
# The consumer key and secret will be generated for you after
consumer_key="rlCkqsZTQVtPbn1eNcp8W4nRB"
consumer_secret="Ek4XgMEjz4yVxYnwtM2a2J1yodMjXY5mwcRJJnJDvcGh3FODqC"

# After the step above, you will be redirected to your app's page.
# Create an access token under the the "Your access token" section
access_token="122083690-XdJNW0LsR9TzZEGjAojvCqYW3LgZS2ZNZyH1mgWU"
access_token_secret="moCTCORNupEB16iJqTXScJmUGjbPmJHNfh7FBuXsvtHRA"

In [399]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [400]:
place = api.geo_search(query=STATE, granularity="admin")
place_id = place[0].id

In [401]:
only_tweets = []
topic_list = []

new_search = str(SEARCH_WORD) + " -filter:retweets" 
tweets = tw.Cursor(api.search, 
                       q=new_search+ ' place:%s' % place_id,
                       lang="en",
                       since="2019-11-08").items(1000)
#only_tweets.append(tweets)
# topic_list.append(key)


In [402]:
# Getting the tweets into an array which would be then be loaded to dataframe
full_arr= []

for tweet in tweets:
    #print(tweet)
    arr = [tweet.user.name, tweet.place.name,tweet.place.full_name, tweet.user.followers_count, tweet.created_at, tweet.retweet_count, tweet.text]
    print(arr)
    full_arr.append(arr)

['DeShayTheGreat 🐐', 'Minneapolis', 'Minneapolis, MN', 759, datetime.datetime(2019, 12, 9, 5, 6, 31), 1, 'Me at target https://t.co/2kQDvmy4Gq']
['St. Paul Pork Product', 'St Paul', 'St Paul, MN', 1388, datetime.datetime(2019, 12, 9, 4, 36, 30), 0, '@Kittyapolis Thank you! It’s vintage. Target, circa 2014!']
['kali. 🐌', 'Bloomington', 'Bloomington, MN', 1737, datetime.datetime(2019, 12, 9, 4, 14, 47), 0, 'i bought it at target :) https://t.co/AJBKHF7QUz']
['Kirsten Brown', 'St Michael', 'St Michael, MN', 690, datetime.datetime(2019, 12, 9, 3, 26, 25), 0, '@MinnesotaBen I do this, except for Little Drummer Boy — any version. It starts on 12:01 on Black Friday. I was out… https://t.co/Bu5V8RpuqW']
['Steve Medin', 'Minnesota', 'Minnesota, USA', 265, datetime.datetime(2019, 12, 9, 1, 53, 43), 0, '@dcherring @pattonoswalt @GerryDuggan @HamillHimself Stay on target... stay on target!']
['Kate', 'Chanhassen', 'Chanhassen, MN', 686, datetime.datetime(2019, 12, 9, 1, 49, 40), 0, "My in-laws see

In [403]:
final_dataframe = pd.DataFrame(full_arr,columns = ['user', "place","place_full_name", "follower count", "created at","retweets", "text"])
final_dataframe['sentiment'] = round(final_dataframe.text.apply(lambda x: TextBlob(x).sentiment[0]),2)
final_dataframe

,user,place,place_full_name,follower count,created at,retweets,text,sentiment
0,DeShayTheGreat 🐐,Minneapolis,"Minneapolis, MN",759,2019-12-09 05:06:31,1,Me at target https://t.co/2kQDvmy4Gq,0.00
1,St. Paul Pork Product,St Paul,"St Paul, MN",1388,2019-12-09 04:36:30,0,"@Kittyapolis Thank you! It’s vintage. Target, ...",0.00
2,kali. 🐌,Bloomington,"Bloomington, MN",1737,2019-12-09 04:14:47,0,i bought it at target :) https://t.co/AJBKHF7QUz,0.50
3,Kirsten Brown,St Michael,"St Michael, MN",690,2019-12-09 03:26:25,0,"@MinnesotaBen I do this, except for Little Dru...",-0.18
4,Steve Medin,Minnesota,"Minnesota, USA",265,2019-12-09 01:53:43,0,@dcherring @pattonoswalt @GerryDuggan @HamillH...,0.00
...,...,...,...,...,...,...,...,...
146,Living w/ Positivity,Edina,"Edina, MN",399,2019-11-30 18:56:02,0,"I'm at Target in Edina, MN https://t.co/9ehfiC...",0.00
147,claire 🍉,Andover,"Andover, MN",268,2019-11-30 18:38:52,0,@facetoward target is my happy place😔✌️ at lea...,0.25
148,ben/ito,Minneapolis,"Minneapolis, MN",1405,2019-11-30 15:18:31,0,"Okay, but target, you have betrayed me. \n\nI ...",0.50
149,Well GameDay Loved Us,Prior Lake,"Prior Lake, MN",167,2019-11-30 11:21:15,3,Just took my 16 year old to his 5:15 am Target...,0.10


In [404]:
final_dataframe['topic'] = SEARCH_WORD

In [405]:
def sentiment_bucket(value):
    sentiment = ""
    if value > 0.02:
        sentiment = "Positive"
    elif value < -0.02:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return sentiment

final_dataframe['SentimentBucket'] = final_dataframe['sentiment'].apply(sentiment_bucket)

def influencer_level(count):
    level = ""
    if count > 1000000:
        level = "Key"
    elif count > 100000:
        level = "Big"
    elif count > 10000:
        level = "Medium"
    elif count > 5000:
        level = "Local"
    else:
        level = "Not"
    return level

final_dataframe['influencerLevel'] = final_dataframe["follower count"].apply(influencer_level)


In [406]:
final_dataframe.head(5)

,user,place,place_full_name,follower count,created at,retweets,text,sentiment,topic,SentimentBucket,influencerLevel
0,DeShayTheGreat 🐐,Minneapolis,"Minneapolis, MN",759,2019-12-09 05:06:31,1,Me at target https://t.co/2kQDvmy4Gq,0.00,Target,Neutral,Not
1,St. Paul Pork Product,St Paul,"St Paul, MN",1388,2019-12-09 04:36:30,0,"@Kittyapolis Thank you! It’s vintage. Target, ...",0.00,Target,Neutral,Not
2,kali. 🐌,Bloomington,"Bloomington, MN",1737,2019-12-09 04:14:47,0,i bought it at target :) https://t.co/AJBKHF7QUz,0.50,Target,Positive,Not
3,Kirsten Brown,St Michael,"St Michael, MN",690,2019-12-09 03:26:25,0,"@MinnesotaBen I do this, except for Little Dru...",-0.18,Target,Negative,Not
4,Steve Medin,Minnesota,"Minnesota, USA",265,2019-12-09 01:53:43,0,@dcherring @pattonoswalt @GerryDuggan @HamillH...,0.00,Target,Neutral,Not


In [407]:
final_dataframe = final_dataframe[['place','created at','sentiment','topic','SentimentBucket','influencerLevel','follower count']]
final_dataframe.columns = ['place','created_at','sentiment','topic','SentimentBucket','influencerLevel','follower_count']

In [ ]:
interest_over_time = interest_over_time[['date', 'Target']]
interest_over_time.columns = ['date_search','title']

In [410]:
import pandas_gbq
import pandas

pandas_gbq.to_gbq(final_dataframe,project_id='trends-msba', destination_table = 'dataset_1.senti_trends_influence' ,if_exists= 'replace')

1it [00:03,  3.51s/it]
